# 快速开始

单击顶部栏上的`🚀` -&gt; `Binder`按钮能够在线运行此示例。

这个例子将展示：

- 如何通过节点连接到区块链
- 如何创建和使用`account`
- 如何与合约交互，如何发送交易

## 连接到节点

我们应该通过一个节点连接到 Conflux 网络。您可以按照[本教程](https://developer.confluxnetwork.org/conflux-doc/docs/get_started/)在本地运行节点，但本地运行节点可能会占用大量资源。因此在这个例子中，我们连接到[Conflux 测试网公共 RPC 端点](https://developer.confluxnetwork.org/sdks-and-tools/en/conflux_rpcs)。

In [1]:
from conflux_web3 import Web3

w3 = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))
assert w3.is_connected()

### 创建一个帐户

在 web3 中，通常，拥有一个帐户意味着知道一个**secret**或**private key** 。这个秘密应该保密，没有人知道。

现在我们将使用`w3.account.create()`创建一个新帐户。此函数从随机密钥生成帐户。由于这个账户是从一个随机密钥生成的，它的余额应该是`0` 。

还支持使用`w3.account.from_key("0x....")`来使用您的密钥（**但不要在不安全的环境中运行它！** ）

In [2]:
acct = w3.account.create()
print(f"account address: {acct.address}")
# WARNING: Don't run the following line in production environment, private key should be kept secret
print(f"account secret key: {acct._private_key.hex()}")
balance = w3.cfx.get_balance(acct.address)
assert balance == 0
print(f"balance for {acct.address}: {balance}")

account address: cfxtest:aaswj28188e35rh1vguksgnuz2xy4f8apye3745zxb
account secret key: 0x54d957b2485980fc2119ccce6480ad0d53219161595424fbf7dc452d59c6bc82
balance for cfxtest:aaswj28188e35rh1vguksgnuz2xy4f8apye3745zxb: 0 Drip


## 通过测试网水龙头合约申领 CFX

因为账户的余额是`0` ，所以它负担不起发送交易所需的**gas** 。但是，Conflux 的[赞助机制](https://forum.conflux.fun/t/conflux-sponsorship-mechanism/12764)允许用户在不支付 gas 的情况下与智能合约进行交互，因此我们可以从[testnet faucet](https://testnet.confluxscan.net/address/cfxtest:acejjfa80vj06j2jgtz9pngkv423fhkuxj786kjr61)领取 CFX。

> 什么是智能合约？智能合约是部署在区块链网络上的程序。它提供接口来完成特定的逻辑。在示例中，调用水龙头合约的`claimCfx`方法将为您提供 1000 个测试网 CFX。

In [3]:
# Firstly we will set `w3.cfx.default_account` to `acct``, after that transactions can be automatically sent.
w3.cfx.default_account = acct

# Next we will interact with testnet Faucet contract
faucet = w3.cfx.contract(name="Faucet")
tx_hash = faucet.functions.claimCfx().transact()

print(f"tx hash is: {tx_hash.hex()}\n"
      f"conflux scan link: https://testnet.confluxscan.net/transaction/{tx_hash.hex()}")

tx hash is: 0xf174cf4be19b33da58504ef850bfd055bd031324135093276abbee1c34df854d
conflux scan link: https://testnet.confluxscan.net/transaction/0xf174cf4be19b33da58504ef850bfd055bd031324135093276abbee1c34df854d


In [4]:
# in Conflux, the transaction will be executed only after it appears on the chain for 5 epoch
# `tx_hash.executed()` is equivalent to `w3.cfx.wait_for_transaction_receipt(tx_hash)`
tx_hash.executed()
print(f"balance for {acct.address}: {w3.cfx.get_balance(acct.address).to('CFX')}")

balance for cfxtest:aaswj28188e35rh1vguksgnuz2xy4f8apye3745zxb: 1000 CFX


## 将 CFX 发送给其他人

现在我们有大量的测试网 CFX 可以做任何你想做的事情。例如，我们可以将 1 个 CFX 发送到零地址。

In [5]:
# Now acct has CFX
# send 1 CFX to zero address
w3.cfx.send_transaction({
    "to": w3.address.zero_address(),
    "value": 10**18,
}).executed()
print(f"balance for {acct.address}: {w3.cfx.get_balance(acct.address).to('CFX')}")

balance for cfxtest:aaswj28188e35rh1vguksgnuz2xy4f8apye3745zxb: 998.999979 CFX
